In [8]:
import os
import cv2
import pickle
import numpy as np
from glob import glob
from insightface.app.common import Face
from insightface.model_zoo import model_zoo
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [9]:
# detection and recognition models
det_model_path = './models/buffalo_l/det_10g.onnx'  # RetinaFace-10GF for detection
rec_model_path = './models/buffalo_l/w600k_r50.onnx'  # ResNet50@WebFace600K for recognition

det_model = model_zoo.get_model(det_model_path)
rec_model = model_zoo.get_model(rec_model_path)

det_model.prepare(ctx_id=0, input_size=(640, 640), det_thres=0.5)
rec_model.prepare(ctx_id=0, input_size=(640, 640), det_thres=0.5)

2025-05-19 07:14:59.314160398 [E:onnxruntime:Default, provider_bridge_ort.cc:2195 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1778 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.12: cannot open shared object file: No such file or directory

2025-05-19 07:14:59.314185332 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:1055 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Require cuDNN 9.* and CUDA 12.*. Please install all dependencies as mentioned in the GPU requirements page (https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements), make sure they're in the PATH, and that your GPU is supported.
2025-05-19 07:14:59.535439799 [E:onnxruntime:Default, provider_bridge_ort.cc:2195 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


In [10]:
def extract_face_embeddings():
    embeddings_path = './uploads/known_embeddings.npy'  # embeddings path
    names_path = './uploads/known_names.pkl'  # names path

    if os.path.exists(embeddings_path) and os.path.exists(names_path):
        known_embeddings = np.load(embeddings_path)
        with open(names_path, 'rb') as f:
            known_names = pickle.load(f)
    else:
        known_embeddings = np.array([]).reshape(0, 512)  # empty array for embeddings
        known_names = []  # empty list for names

    # list all directories in the 'Dataset' folder
    person_dirs = [d for d in os.listdir('./dataset') if os.path.isdir(os.path.join('./dataset', d))]
    print("Extracting embeddings...")

    for person_name in person_dirs:
        directory = f'./dataset/{person_name}'
        img_paths = glob(f'{directory}/*.jpg')
        new_embeddings = []  # list to store embeddings for each person

        for idx, img_path in enumerate(img_paths):
            img = cv2.imread(img_path)
            if img is None:
                continue
            bboxes, kpss = det_model.detect(img, max_num=0, metric='default')
            if len(bboxes) == 0:
                continue

            # assuming the first bounding box is the correct one
            bbox = bboxes[0, :4]
            det_score = bboxes[0, 4]
            kps = kpss[0]
            face = Face(bbox=bbox, kps=kps, det_score=det_score)
            rec_model.get(img, face)  # face embedding using the recognition model

            if hasattr(face, 'normed_embedding'):
                new_embeddings.append(face.normed_embedding)  # append the embedding
            else:
                print(f"Failed to extract embedding for {img_path}")

        # if new embeddings are found for the person, add them to the list
        if new_embeddings:
            new_embeddings = np.vstack(new_embeddings)
            known_embeddings = np.vstack([known_embeddings, new_embeddings])
            known_names.extend([person_name] * new_embeddings.shape[0])

    # Save embeddings and names
    np.save(embeddings_path, known_embeddings)
    with open(names_path, 'wb') as f:
        pickle.dump(known_names, f)

    # Print all embeddings at the end of the process
    print("All embeddings extracted and saved successfully.")
    print(known_embeddings)

extract_face_embeddings()


Extracting embeddings...
All embeddings extracted and saved successfully.
[[ 0.01218998  0.01847454 -0.0102439  ...  0.08157813  0.0472879
  -0.0733004 ]
 [ 0.02232696 -0.01465663 -0.0834538  ...  0.06656986  0.07133806
  -0.07881003]
 [ 0.03928491  0.00099033 -0.05734017 ...  0.08792619  0.08691135
  -0.07987834]
 ...
 [ 0.03148594  0.07301585  0.07003179 ... -0.02915545 -0.01492532
   0.02969946]
 [ 0.034522    0.08771636  0.08340845 ... -0.07423431 -0.01619307
   0.04900846]
 [ 0.02049444  0.08852427  0.09765343 ... -0.05429428 -0.01989821
   0.04645144]]


In [17]:
def load_embeddings():
    embeddings_path = './uploads/known_embeddings.npy'
    names_path = './uploads/known_names.pkl'

    if os.path.exists(embeddings_path) and os.path.exists(names_path):
        known_embeddings = np.load(embeddings_path)
        with open(names_path, 'rb') as f:
            known_names = pickle.load(f)
        return known_embeddings, known_names
    else:
        print("No saved embeddings found.")
        return None, None

In [12]:


def find_match(embedding, known_embeddings, known_names, threshold=0.5):
    scores = np.dot(embedding, known_embeddings.T) # cosine similarity between embeddings
    scores = np.clip(scores, 0., 1.)
    idx = np.argmax(scores) #  index of the highest score
    score = scores[idx]
    # return name if the score exceeds the threshold, otherwise return 'Unknown'
    return known_names[idx] if score > threshold else 'Unknown'



In [13]:
def face_recognition(known_embeddings, known_names, threshold=0.5):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Unable to open camera.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        bboxes, kpss = det_model.detect(frame, max_num=0, metric='default')
        if len(bboxes) > 0:
            for i in range(len(bboxes)):
                bbox = bboxes[i, :4]
                kps = kpss[i]
                face = Face(bbox=bbox, kps=kps, det_score=bboxes[i, 4])
                rec_model.get(frame, face)
                test_embedding = face.normed_embedding
                pred_name = find_match(test_embedding, known_embeddings, known_names, threshold)

                x1, y1, x2, y2 = map(int, bbox)
                face_crop = frame[y1:y2, x1:x2]

                label = f"{pred_name}"

                # Set color
                if pred_name == 'Unknown':
                    color = (0, 0, 255)
                elif pred_name == 'sayan':
                    color = (215, 168, 150)
                else:
                    color = (70, 255, 20)

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 2)

        cv2.imshow("Face Recognition + Mask Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [18]:
# %%capture
known_embeddings, known_names = load_embeddings()
if known_embeddings is not None and known_names is not None:
    face_recognition(known_embeddings, known_names)

